# Genetic Algorithms

### A whirlwind tour of bio-inspired computing algorithms

#### Quentin Caudron

Number of possible solutions is enormous ( cannot search the space )

Constraints imposed by paramter combinations or in the solution lead to difficulty finding solutions or searching the space 

